# DC CTC Reform: Child Poverty Reduction by Age Range

Analysis of child poverty impact for DC CTC reform with:
- $1,000 per child
- 10% phase-out ($100 reduction per $1,000 over income threshold)
- No cap on number of children
- Lower income thresholds: $50k (Single/HoH/Surviving), $75k (Joint), $37.5k (Separate)

**Baseline**: Current DC CTC repealed (zeroed out)

We'll analyze the percentage reduction in overall child poverty for different age eligibility thresholds.

In [11]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np

In [12]:
# Create baseline with current DC CTC zeroed out
def create_baseline_reform():
    """Zero out the existing DC CTC"""
    return Reform.from_dict(
        {
            "gov.states.dc.tax.income.credits.ctc.amount": {
                "2025-01-01.2100-12-31": 0
            }
        },
        country_id="us",
    )

baseline_reform = create_baseline_reform()
baseline_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=baseline_reform)

In [13]:
def create_reform(age_threshold):
    """
    Create DC CTC reform with:
    - $1,000 per child
    - 10% phase-out ($100 per $1,000 over threshold)
    - No cap on children
    - Lower income thresholds
    """
    return Reform.from_dict(
        {
            "gov.states.dc.tax.income.credits.ctc.amount": {
                "2025-01-01.2100-12-31": 1000
            },
            "gov.states.dc.tax.income.credits.ctc.child.age_threshold": {
                "2025-01-01.2100-12-31": age_threshold
            },
            "gov.states.dc.tax.income.credits.ctc.child.child_cap": {
                "2025-01-01.2100-12-31": 999
            },
            "gov.states.dc.tax.income.credits.ctc.phase_out.amount": {
                "2025-01-01.2100-12-31": 100
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SINGLE": {
                "2025-01-01.2100-12-31": 50_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.JOINT": {
                "2025-01-01.2100-12-31": 75_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SEPARATE": {
                "2025-01-01.2100-12-31": 37_500
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.HEAD_OF_HOUSEHOLD": {
                "2025-01-01.2100-12-31": 50_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SURVIVING_SPOUSE": {
                "2025-01-01.2100-12-31": 50_000
            }
        },
        country_id="us",
    )

In [14]:
def calculate_child_poverty(sim):
    """
    Calculate overall child poverty rate (children under 18).
    Returns poverty rate, count in poverty, and total children.
    """
    # Get person-level data
    age = sim.calculate("age", period=2025)
    is_in_poverty = sim.calculate("person_in_poverty", period=2025)
    
    # Filter for children under 18
    mask = age < 18
    
    # Calculate poverty count and total children
    # The simulation data is already weighted, so we just count/sum
    children_in_poverty = is_in_poverty[mask].sum()
    total_children = mask.sum()
    
    # Calculate poverty rate
    poverty_rate = children_in_poverty / total_children if total_children > 0 else 0
    
    return {
        "poverty_rate": poverty_rate,
        "children_in_poverty": children_in_poverty,
        "total_children": total_children
    }

## Baseline Child Poverty Rate

Baseline: Current DC CTC repealed (zeroed out)

In [15]:
# Calculate baseline child poverty rate (with DC CTC zeroed out)
baseline_poverty = calculate_child_poverty(baseline_sim)

print("BASELINE CHILD POVERTY RATE (current DC CTC repealed)")
print("="*70)
print(f"Children under 18: {baseline_poverty['poverty_rate']*100:.2f}%")
print(f"Children in poverty: {baseline_poverty['children_in_poverty']:,.0f}")
print(f"Total children: {baseline_poverty['total_children']:,.0f}")

BASELINE CHILD POVERTY RATE (current DC CTC repealed)
Children under 18: 21.13%
Children in poverty: 27,758
Total children: 131,386


## Reform Impact Analysis

Analyze reforms with different age eligibility thresholds (6, 8, 12, 18).

Each reform uses:
- $1,000 per child
- 10% phase-out
- No child cap
- Lower income thresholds

In [16]:
# Define age thresholds to analyze
age_thresholds = [6, 8, 12, 18]

# Store baseline values for comparison
baseline_rate = baseline_poverty['poverty_rate']

# Analyze each reform
results = []

for age_threshold in age_thresholds:
    print(f"\nAnalyzing reform: Eligible children under age {age_threshold}")
    
    # Create reform simulation
    reform = create_reform(age_threshold)
    reform_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=reform)
    
    # Calculate poverty rate under reform
    reform_poverty = calculate_child_poverty(reform_sim)
    reform_rate = reform_poverty['poverty_rate']
    
    # Calculate cost
    reform_ctc = reform_sim.calculate("dc_ctc", period=2025, map_to="household")
    total_cost = reform_ctc.sum()
    
    # Calculate percentage reduction
    if baseline_rate > 0:
        pct_reduction = ((baseline_rate - reform_rate) / baseline_rate) * 100
    else:
        pct_reduction = 0
    
    absolute_reduction = (baseline_rate - reform_rate) * 100  # in percentage points
    
    results.append({
        "Age Eligibility": f"Under {age_threshold}",
        "Baseline Poverty Rate": f"{baseline_rate*100:.2f}%",
        "Reform Poverty Rate": f"{reform_rate*100:.2f}%",
        "Absolute Reduction (pp)": f"{absolute_reduction:.2f}",
        "Percentage Reduction": f"{pct_reduction:.1f}%",
        "Annual Cost": f"${total_cost:,.0f}",
        # Numeric values for sorting
        "Age (Numeric)": age_threshold,
        "Baseline Rate (Numeric)": baseline_rate,
        "Reform Rate (Numeric)": reform_rate,
        "Pct Reduction (Numeric)": pct_reduction,
        "Cost (Numeric)": total_cost
    })
    
    print(f"  Poverty rate: {baseline_rate*100:.2f}% → {reform_rate*100:.2f}% ({pct_reduction:.1f}% reduction)")
    print(f"  Annual cost: ${total_cost:,.0f}")

print("\nAnalysis complete!")


Analyzing reform: Eligible children under age 6
  Poverty rate: 21.13% → 20.86% (1.3% reduction)
  Annual cost: $25,164,361

Analyzing reform: Eligible children under age 8
  Poverty rate: 21.13% → 20.86% (1.3% reduction)
  Annual cost: $33,342,733

Analyzing reform: Eligible children under age 12
  Poverty rate: 21.13% → 20.33% (3.8% reduction)
  Annual cost: $53,009,724

Analyzing reform: Eligible children under age 18
  Poverty rate: 21.13% → 19.76% (6.4% reduction)
  Annual cost: $82,271,789

Analysis complete!


## Results Summary

Overall child poverty reduction (children under 18) for each age eligibility threshold.

In [17]:
# Create DataFrame
df = pd.DataFrame(results)

# Display full results table
print("\n" + "="*120)
print("DC CTC REFORM: CHILD POVERTY REDUCTION BY AGE ELIGIBILITY")
print("Reform Parameters: $1,000/child, 10% phase-out, No child cap")
print("Baseline: Current DC CTC repealed")
print("="*120)
print(df[['Age Eligibility', 'Baseline Poverty Rate', 'Reform Poverty Rate', 
          'Absolute Reduction (pp)', 'Percentage Reduction', 'Annual Cost']].to_string(index=False))
print("="*120)


DC CTC REFORM: CHILD POVERTY REDUCTION BY AGE ELIGIBILITY
Reform Parameters: $1,000/child, 10% phase-out, No child cap
Baseline: Current DC CTC repealed
Age Eligibility Baseline Poverty Rate Reform Poverty Rate Absolute Reduction (pp) Percentage Reduction Annual Cost
        Under 6                21.13%              20.86%                    0.27                 1.3% $25,164,361
        Under 8                21.13%              20.86%                    0.27                 1.3% $33,342,733
       Under 12                21.13%              20.33%                    0.79                 3.8% $53,009,724
       Under 18                21.13%              19.76%                    1.36                 6.4% $82,271,789


## Cost-Effectiveness Analysis

Compare the cost per percentage point reduction in child poverty.

In [18]:
# Calculate cost per percentage point reduction
print("\nCOST-EFFECTIVENESS COMPARISON")
print("="*100)

cost_effectiveness = []
for _, row in df.iterrows():
    if row['Pct Reduction (Numeric)'] > 0:
        cost_per_pp = row['Cost (Numeric)'] / abs((row['Baseline Rate (Numeric)'] - row['Reform Rate (Numeric)']) * 100)
        cost_effectiveness.append({
            "Age Eligibility": row['Age Eligibility'],
            "Percentage Reduction": row['Percentage Reduction'],
            "Annual Cost": row['Annual Cost'],
            "Cost per pp Reduction": f"${cost_per_pp:,.0f}"
        })

cost_df = pd.DataFrame(cost_effectiveness)
print(cost_df.to_string(index=False))
print("="*100)


COST-EFFECTIVENESS COMPARISON
Age Eligibility Percentage Reduction Annual Cost Cost per pp Reduction
        Under 6                 1.3% $25,164,361           $93,190,797
        Under 8                 1.3% $33,342,733          $123,477,635
       Under 12                 3.8% $53,009,724           $66,697,531
       Under 18                 6.4% $82,271,789           $60,376,900


## Key Findings

In [19]:
print("\nKEY FINDINGS")
print("="*80)
print(f"\nBaseline child poverty rate (DC CTC repealed): {baseline_rate*100:.2f}%")
print(f"\nChild poverty reduction by reform:")
for _, row in df.iterrows():
    print(f"  {row['Age Eligibility']:10s}: {row['Percentage Reduction']:>6s} reduction (costs {row['Annual Cost']})")

# Find most cost-effective
if len(cost_effectiveness) > 0:
    best_reform = min(cost_effectiveness, key=lambda x: float(x['Cost per pp Reduction'].replace('$','').replace(',','')))
    print(f"\nMost cost-effective: {best_reform['Age Eligibility']} ({best_reform['Cost per pp Reduction']} per percentage point reduction)")
print("="*80)


KEY FINDINGS

Baseline child poverty rate (DC CTC repealed): 21.13%

Child poverty reduction by reform:
  Under 6   :   1.3% reduction (costs $25,164,361)
  Under 8   :   1.3% reduction (costs $33,342,733)
  Under 12  :   3.8% reduction (costs $53,009,724)
  Under 18  :   6.4% reduction (costs $82,271,789)

Most cost-effective: Under 18 ($60,376,900 per percentage point reduction)


In [20]:
# Export results to CSV
csv_filename = "dc_ctc_child_poverty_by_age.csv"
df[['Age Eligibility', 'Baseline Poverty Rate', 'Reform Poverty Rate', 
    'Absolute Reduction (pp)', 'Percentage Reduction', 'Annual Cost']].to_csv(csv_filename, index=False)
print(f"\nResults exported to {csv_filename}")


Results exported to dc_ctc_child_poverty_by_age.csv
